In [1]:
!pip install openai python-dotenv



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import os
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv

# Load environment variables
load_dotenv()
_ = load_dotenv(find_dotenv())

# Initialize OpenAI client
client = OpenAI(api_key=os.environ.get('open_api_key'))


In [38]:
class TranscriptProcessor:
    def __init__(self, file_path):
        self.file_path = file_path

    def load_transcript(self):
        try:
            with open(self.file_path, 'r', encoding='utf-8') as file:
                transcript = file.read()
            return transcript
        except FileNotFoundError:
            raise FileNotFoundError(f"Transcript file not found at {self.file_path}")
        except Exception as e:
            raise Exception(f"Error loading transcript: {e}")

# Define transcript file path
input_path = "C:/Users/INSIGHT/Desktop/4th Year Sem 2/ChatGPTAPI/TranscriptReader/patientenv/transcript.txt"

# Load transcript
processor = TranscriptProcessor(input_path)
transcript_text = processor.load_transcript()

print("Transcript Loaded Successfully!")


Transcript Loaded Successfully!


In [58]:
class ApiInteraction:
    def __init__(self):
        # Retrieve the API key from environment variables
        self.api_key = os.environ.get('open_api_key')
        if not self.api_key:
            raise ValueError("API key is missing from the environment variables")
        
        # Set the OpenAI API key
        client.api_key = self.api_key

    def generate_patient_letter(self, transcript):
        """
        Generate a detailed patient letter directly from the transcript.
        Extracts details and fills in a structured letter format.
        """
        prompt = f"""
        You are a professional dentist tasked with writing a follow-up letter to a patient based on the consultation transcript. 
        Extract relevant patient details (like name, symptoms, findings, and treatment recommendations) 
        and format the letter as follows:

        Patient Follow-Up Letter Template:

        Dear [Patient Name],

        Thank you for visiting our practice on [most recent visit] if the date is not mentioned then add some suitable sentence in the place holder not regarding date but something generally. It was a pleasure meeting you and addressing your dental concerns.

        Summary of Findings:
        - Main Symptoms: [List main symptoms]
        - Clinical Observations: [Describe clinical observations, including any inflammation, decay, plaque, or bone loss]
        - X-ray Findings: [Summarize relevant X-ray results]

        Treatment Plan:
        - Recommended Procedures: [List recommended treatments, e.g., filling replacement, deep cleaning, root canal, etc.]
        - Oral Hygiene Recommendations: [Provide specific hygiene advice, such as switching toothbrushes or flossing more regularly]
        - Lifestyle Adjustments: [Suggest lifestyle changes, like reducing smoking or modifying diet]

        Next Steps:
        - Scheduled Appointment: [Provide the next appointment date or placeholder if missing]
        - Interim Advice:[Give advice or say "monitor symptoms and contact us if needed"]

        Your oral health is our top priority, and we are committed to supporting you every step of the way. If you have any questions or experience worsening symptoms, please don’t hesitate to contact us.

        Best wishes for your continued dental health,

        If dentist details are found in the transcript, replace the following with the extracted details. Otherwise, default to:

        Chairsyde
        info@chairsyde.com
        +1 (555) 123-4567

        Transcript:
        {transcript}

        Fill in this structure with the extracted details, and leave placeholders if certain information is not available.
        """
        
        # Call the OpenAI API to generate the letter based on the prompt
        completion = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "system", "content": prompt}],
            temperature=0.2,
            max_tokens=500,
        )

        return completion.choices[0].message.content


# Generate the letter
api_instance = ApiInteraction()
patient_letter = api_instance.generate_patient_letter(transcript_text)

# Print the generated letter
print("Letter Generated Successfully!\n")
print(patient_letter)

Letter Generated Successfully!

Dear Mark,

Thank you for visiting our practice. It was a pleasure meeting you and addressing your dental concerns.

Summary of Findings:
- Main Symptoms: Crooked teeth, mild crowding in the lower front teeth, and a slight rotation on your upper right canine.
- Clinical Observations: Your gums look good, with BP scores of zeros all around, which is excellent. You have a small composite filling on your upper right six, which appears sound.
- X-ray Findings: Your bone levels are healthy, and there are no signs of decay. The crowding we see is mild to moderate.

Treatment Plan:
- Recommended Procedures: Clear aligners, such as Invisalign, are recommended for your case. This treatment starts with a digital scan of your teeth, which allows us to create a custom plan for your alignment. Small, tooth-colored attachments may be placed on some teeth to assist with movement.
- Oral Hygiene Recommendations: Continue maintaining your current oral hygiene routine.
- 

In [59]:
class LetterFormatter:
    @staticmethod
    def format_letter(letter):
        """
        Format the generated letter to ensure proper structure and style,
        making it look professional and branded for Chairsyde Dental Clinic.
        """
        # Define the clinic header and footer
        clinic_header = """
                                                **********************************************
                                                *     Chairsyde Dental Clinic                *
                                                *     Your Smile, Our Priority!              *
                                                *     www.chairsyddentalclinic.com           *
                                                *     Call Us: (555) 123-4567                *
                                                **********************************************\n"""
        
        clinic_footer = """
                                        *************************************************************************
                                        * Thank you for trusting Chairsyde Dental Clinic with your oral health! *
                                        * If you have any questions, don't hesitate to contact us.              *
                                        ************************************************************************* """

        # Apply header and footer to the letter
        formatted_letter = clinic_header + letter + clinic_footer
        
        # Enhance formatting: make sections bold and clearly separated
        formatted_letter = formatted_letter.replace("Summary of Findings:", "SUMMARY OF FINDINGS:\n")
        formatted_letter = formatted_letter.replace("Treatment Plan:", "TREATMENT PLAN:\n")
        formatted_letter = formatted_letter.replace("Next Steps:", "NEXT STEPS:\n")
        
        # Add some spacing between sections for clarity
        formatted_letter = formatted_letter.replace("\n", "\n")

        return formatted_letter

# Format letter
formatted_patient_letter = LetterFormatter.format_letter(patient_letter)

print("Letter Formatted Successfully!")
print(formatted_patient_letter)


Letter Formatted Successfully!

                                                **********************************************
                                                *     Chairsyde Dental Clinic                *
                                                *     Your Smile, Our Priority!              *
                                                *     www.chairsyddentalclinic.com           *
                                                *     Call Us: (555) 123-4567                *
                                                **********************************************
Dear Mark,

Thank you for visiting our practice. It was a pleasure meeting you and addressing your dental concerns.

SUMMARY OF FINDINGS:

- Main Symptoms: Crooked teeth, mild crowding in the lower front teeth, and a slight rotation on your upper right canine.
- Clinical Observations: Your gums look good, with BP scores of zeros all around, which is excellent. You have a small composite filling 

In [32]:
output_path = "C:/Users/INSIGHT/Desktop/4th Year Sem 2/ChatGPTAPI/TranscriptReader/patientenv/patient_letter.txt"

with open(output_path, "w", encoding="utf-8") as file:
    file.write(formatted_patient_letter)

print(f"Patient letter saved to {output_path}")


Patient letter saved to C:/Users/INSIGHT/Desktop/4th Year Sem 2/ChatGPTAPI/TranscriptReader/patientenv/patient_letter.txt
